# Pytorch

Pytorch es una biblioteca creada por Facebook para hacer "programación diferencial".

Una biblioteca de programación diferencial es un sistema en donde haces operaciones y el sistema se "acuerda" de todas las operaciones que hiciste, para poder sacar derivadas.

Por ejemplo, digamos que tienes `a = 5` y `b = 3` y yo le pregunto a python `a*b`. Sin problema me contestará `15`, pero una vez que hace la operación, se olvida de donde venía ese `15`. En programación diferencial no: se acuerda  de que tenía dos variables y que las multipliqué para obtener el 15, así que si luego le pregunto a pytorch por la derivada de `a*b` con respecto a `a` y con respecto a `b`, lo puede sacar (evaluado en `(5,3)`, claro!).

Además, pytorch hace álgebra lineal, claro, y programación diferencial con álgebra lineal.

La unidad básica en pytorch son los *tensores*, que son la generalización de número, vector, matriz, etc. Si hay físicos en la audiencia, me disculpo de antemano (de parte de toda la comunidad de deep learning) por llamarle tensores a esas cosas :)

Algunas notas:

- pytorch está pensado para hacer redes neuronales, pero no es necesario querer hacer redes neuronales para usarlo!
- pytorch es extremadamente rápido. En realidad cuando le dices que multiplique matrices o lo que sea, llama a código hiper-eficiente de C++.
- Puedes usar pytorch directamente en C++, pero no ganas mucho si tus cuellos de botella son las operaciones en sí que estás haciendo.
- Puedes usar pytorch en la GPU, lo cual hace mucho más rápidas las operaciones. Incluso puedes usar muchas GPUs, o un cluster de computadoras cada una con sus GPUs, etc.
- Trae muchas funciones de ayuda para leer imágenes/audio/texto, etc. etc. Es una biblioteca muy completa. Nosotros nos enfocaremos exclusivamente a la manipulación de tensores.

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

In [ ]:
a = torch.tensor([5.])

In [ ]:
b = torch.tensor([3.])

In [ ]:
a*b

Ahora, a esos tensores no les dije que necesitaban gradientes, así que no sabe sacar derivadas ni nada. Para poder sacar derivadas, necesito decirlo a la hora de crearlos.

In [ ]:
a = torch.tensor([5.],requires_grad=True)

In [ ]:
b = torch.tensor([3.],requires_grad=True)

In [ ]:
a*b

In [ ]:
(a*b).backward() # esto significa "saca derivadas de a*b con respecto a todo"

In [ ]:
a

In [ ]:
a.grad # eso fue la derivada parcial de (a*b) con respecto a, que efectivamente es 3!

In [ ]:
b.grad

Ahora, torch puede trabajar con tensores de todo tipo (enteros, boleanos, etc. etc.), pero claro, solo puede sacar derivadas de los floats.

In [ ]:
A = torch.tensor([[1,2.0,3,4],[5,6,7,8]])

In [ ]:
A

## Propiedades

In [ ]:
A.shape

In [ ]:
A.dtype

In [ ]:
A.device

In [ ]:
A=A.to('cuda')

## Operaciones

In [ ]:
3*A

In [ ]:
A+A

In [ ]:
torch.sin(A)

In [ ]:
A.T

In [ ]:
A@A.T

In [ ]:
torch.arange(10)

In [ ]:
torch.arange(10,20,2)

## Generar aleatorios

In [ ]:
B = torch.rand(3,5,5); B

In [ ]:
B.to(A.device)

In [ ]:
A.device

In [ ]:
B=torch.randn(4,3,2) # normal con media 0 y desviación estándar 1

In [ ]:
B

## Indizar

In [ ]:
A

In [ ]:
A[0]

In [ ]:
A[0,1]

In [ ]:
A[:,0]

In [ ]:
A[1,:] # equivalente a A[1], claro

In [ ]:
A[:2,:1]

In [ ]:
A[:,None,:]

In [ ]:
A[:,None,:].shape

## Broadcasting

Una cosa súper padre y súper confusa a veces es el concepto de broadcasting. Si una operación entre tensores no se puede hacer porque las dimensiones no corresponden, a veces se puede hacer: aumentando dimensiones de tamaño "1" y repitiendo. Vamos a ver un ejemplo.

In [ ]:
A + 3

In [ ]:
A + torch.tensor([3])

In [ ]:
torch.tensor([3]).shape

In [ ]:
A

In [ ]:
A + torch.tensor([10,20,30,40])

In [ ]:
torch.tensor([10,20,30,40]).shape

In [ ]:
A.shape

Ejercicio: Súmale 10 al primer renglón y 20 al segundo renglón. Escribe poquito.

In [ ]:
A + torch.tensor([10,20])[:,None]

Ejercicio: Crea un tensor de 10x5x8 (e.g. 10 matrices de 5x8) y luego multiplica la primera matriz por 0, luego la segunda por 1, etc.

## Visualizar

In [ ]:
import torchvision as tv
import numpy as np
import matplotlib.pyplot as plt

def torchimg2numpy(t):
    return np.transpose(t.detach().cpu().numpy(),(1,2,0))

def show_tensor_as_image(tensor, ncols=5, figsize=10, title = ""):
    plt.figure(figsize=(figsize,figsize))
    plt.axis("off")
    plt.title(title)
    plt.imshow(np.transpose(tv.utils.make_grid(tensor.detach().cpu()[:ncols*ncols], nrow=ncols, padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
show_tensor_as_image(torch.randn(3,128,128))

`cat`, `stack`, `repeat`

In [ ]:
A

In [ ]:
torch.cat((A,A),dim=1)

In [ ]:
torch.cat((A,A),dim=0)

In [ ]:
torch.stack((A,A,A))

In [ ]:
torch.stack((A,A,A)).shape

In [ ]:
A

In [ ]:
A.repeat(5,3)

In [ ]:
A.repeat_interleave(6,0)

`view`, `reshape`

In [ ]:
A.shape

In [ ]:
A

In [ ]:
A.view(4,2)

In [ ]:
A.reshape(4,2)

Parecen lo mismo, pero internamente no los acomoda igual en memoria. view sólo cambia las dimensiones, reshape mueve la memoria apropiadamenet para que quede contigua la cosa

In [ ]:
A.view(1,8)

In [ ]:
A.reshape(1,8)

Máscaras

In [ ]:
A

In [ ]:
(A < 3)

In [ ]:
A[A<3]

In [ ]:
A[A%2 == 0]

Fíjate que lo convierte a un tensor de dimensión 1

In [ ]:
A[A%2 == 0] = 99.

In [ ]:
A = torch.tensor([[1,2.,3,4],[5,6,7,8]],requires_grad=True)

In [ ]:
A = A.cuda()

In [ ]:
torch.zeros_like(A)

In [ ]:
torch.ones_like(A)

Lo mejor de las variantes "_like" es que los pone en el mismo dispositivo que el original (gpu, cpu)

In [ ]:
torch.rand_like(A)

In [ ]:
torch.rand_like(A.float())

In [ ]:
A = A.float()

In [ ]:
A

In [ ]:
B = A/2

In [ ]:
B[B<=2] = 0

In [ ]:
A

In [ ]:
(A<=2).float()*A

## Indizar avanzado

In [ ]:
A = torch.tensor([[1,2,3,4],[5,6,7,8.]])

In [ ]:
A[:,:-1]

In [ ]:
A[:,torch.tensor([1,3])]

In [ ]:
A = torch.rand(3,3,3,3,3,3,3,3,3,2)

In [ ]:
A

In [ ]:
torch.arange(4,7,2)

In [ ]:
A[torch.arange(2),torch.tensor([0,2])]

In [ ]:
A[:,torch.tensor([0,2])]

In [ ]:
A = A.reshape(2,1,1,4,1)

In [ ]:
A.shape

In [ ]:
A

In [ ]:
A.squeeze()

In [ ]:
A.squeeze().shape

In [ ]:
A.shape

In [ ]:
A.unsqueeze(0).shape

In [ ]:
A[...,None,:].shape

## Optimización numérica

Si tienes derivadas parciales, puedes encontrar mínimos (o máximos) locales fácilmente. Pytorch trae optimizadores que hacen esto por ti.

Por ejemplo, digamos que queremos encontrar un mínimo local de $x^2-10x + 1$ o lo que sea (agregale senos, cosenos, y otras cosas complicadas si quieres hacer algo que no podrías hacer analíticamente). 

In [ ]:
def f(x): 
    return x**2 - 10*x + 1

In [ ]:
x = torch.randn(1)*10

x = x.requires_grad_(True)

In [ ]:
f(x)

In [ ]:
from torch import optim
from tqdm import tqdm

In [ ]:
optimizer = optim.SGD([x], lr=0.01, momentum=0.9)

In [ ]:
num_steps = 1000
for step in tqdm(range(num_steps)):
    optimizer.zero_grad()
    loss = f(x)
    loss.backward()
    optimizer.step()

In [ ]:
x

**Nota**: Pytorch trae una cantidad **masiva** de cosas. pueden verlo en la documentación. Cualquier cosa que quieras hacer, seguro puedes!

- [Tutoriales](https://pytorch.org/tutorials/)
- [Documentación](https://pytorch.org/docs/stable/index.html)